In [ ]:
"""Data cleaning of used car data set."""
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns
from fuzzywuzzy import fuzz, process
from sklearn.linear_model import LinearRegression

path_to_dataset = "../data/used_car_dataset.csv"

used_car_data = pd.read_csv(
    path_to_dataset,
    dtype={
        "publication_history": str,
        "price_sek": int,
        "price_history": str,
        "entry_year": int,
        "mileage_km": int,
    },
    parse_dates=["publication_datetime"],
    infer_datetime_format="%Y-%m-%d %H:%M:%S",
)
used_car_data = used_car_data.iloc[:, 1:]

used_car_data.reset_index(drop=True, inplace=True)
used_car_data = used_car_data.fillna(value=np.nan)

# General characteristics of data set

Determine general characteristics to get an idea about the data set. 

In [ ]:
print(used_car_data.shape)
used_car_data.head(2)

##  Get an idea about number of unique and missing values

I use this information later to decide which columns to keep and which columns need to be looked at to substitute with data. 

In [ ]:
pd.DataFrame(
    {
        "unique_values": used_car_data.nunique(axis=0),
        "number_missing_values": used_car_data.isna().sum(),
        "percentage_missing_values": used_car_data.isna().sum()
        / used_car_data.shape[0]
        * 100,
        "data_type": used_car_data.dtypes,
    }
)

## Key parameters of numeric values

In [ ]:
used_car_data.describe()

The values in the data set are written in small or capital letters. To align their writting for the data cleaning, I change all writting to small letters.

# Data Cleaning

After I have taken a superficial look at the data set, I will start with the actual data cleaning. 

I will begin with 
- removing duplicates, 
- correcting syntax errors (e.g., remove trailing white spaces and fix typos (using map, pattern match or fuzzy matching), etc.),
- unifying missing values to NA (e.g., they can be "0", "None", "INF", etc.),
- standardizing values (e.g., convert strings to lower case letters, convert numeric values of the same variable to same measurement unit).

This will focus on 
1) reduce the number of different car models,
2) handle missing values,
3) remove outliers.

In [ ]:
used_car_data.drop_duplicates(subset="url", inplace=True)
used_car_data.sort_values(by="publication_datetime", inplace=True)

used_car_data = used_car_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)

used_car_data["price_sek"] = used_car_data["price_sek"].replace(0, np.nan)

## 1) Reduce the number od different car models

Some model names do also include the manufacturer name. To align all model names I remove the manufacturer name.

In [ ]:
for manufacturer in used_car_data["manufacturer"].unique():
    used_car_data["model"].loc[used_car_data["manufacturer"] == manufacturer].replace(
        manufacturer, "", inplace=True
    )

used_car_data["model"] = used_car_data["model"].str.strip()

Some model names are empty but notes have a potential name. Hence, I extract the model name from the first substring of the note string. Here, I assume that each model has a name formed by a single string. 

In [ ]:
model = used_car_data.loc[used_car_data["model"] == "", "note"].str.split().tolist()
used_car_data.loc[used_car_data["model"] == "", "model"] = [part[0] for part in model]

Note: The assumption is faulty when we have model names consisting of a a number and a letter which are separeted by a white space. Hence, I checked manually for cars / manufacturers with these errors.

Probably there are more issues with the model name but the naming convention is not uniform.

In [ ]:
# some BMWs the model contains only the number missing the letter
model_notes = used_car_data.loc[
    used_car_data["manufacturer"] == "bmw", "note"
].str.split()

for idx, note in model_notes.iteritems():
    if note is not np.nan:
        # only models with three numbers have an additional letter
        if len(note[0]) == 1 and len(used_car_data.loc[idx, "model"]) == 3:
            used_car_data.loc[idx, "model"] = (
                used_car_data.loc[idx, "model"] + "" + note[0]
            )

In [ ]:
# some Volvos miss the addition "cross country"
model_notes = used_car_data.loc[
    used_car_data["manufacturer"] == "volvo", "note"
].str.split()
for idx, note in model_notes.iteritems():
    if note is not np.nan:
        if ("cross country" in model) and used_car_data.loc[idx, "model"].isin(
            "cross country"
        ):
            used_car_data.loc[idx, "model"] = (
                used_car_data.loc[idx, "model"] + " cross country"
            )

### Group similar model names 

As there is not uniform naming convention often the same car model is written differently. 
Hence, I group model names based on a fuzzy-logic. 

The thresholds of the fuzzy logic were set manually and derived from a detailed analysis. However, this method is a bit error prone and strongly depends on the chosen thresholds. 

The used thresholds where determined based on an analysis with different manufacturers. Nevertheless, some matches are not found and some are put together wrong.

In [ ]:
def score_matches_of_model_name(unique_model_names, fuzz_scorer, threshold):
    """Score the matches between model names.

    Parameters
    ----------
    unique_model_names : list
        name of car model
    fuzz_scorer : str
        fuzzywuzzy scorer
    threshold : int
        threshold to decide model are similar

    Returns
    -------
    similarity : DataFrame
        similarity score for each model
    """
    score = [
        (x,) + i
        for x in unique_model_names
        for i in process.extract(x, unique_model_names, scorer=fuzz_scorer)
    ]
    similarity = pd.DataFrame(score, columns=["model", "match", "score"])
    # pick only matches that score above threshold and are not a self-match
    similarity = similarity[
        (similarity["score"] >= threshold)
        & (similarity["model"] != similarity["match"])
    ]

    return similarity


def find_and_replace_similar_models(
    manufacturer, scoring_threshold, replacing_threshold, dataset
):
    """Find matches of similar models and replace them.

    The similarity is matched based on two scores as together they provide a better
    matching results.

    Parameters
    ----------
    manufacturer : str
        name of manufacturer
    scoring_threshold : int
        threshold of similarity score to be picked as potential replacement
    replacing_threshold : int
        threshold of similarity score to replace model name with match
    dataset: DataFrame
        dataset of used cars

    Returns
    -------
    dataset: DataFrame
        updated dataset of used cars
    """
    unique_model_names = (
        dataset.loc[dataset["manufacturer"] == manufacturer, "model"].unique().tolist()
    )

    # match similar models based on two scores as sometimes one finds a
    # good match while the other does not
    similarity_sort_ratio = score_matches_of_model_name(
        unique_model_names, fuzz.token_sort_ratio, scoring_threshold
    )
    similarity_set_ratio = score_matches_of_model_name(
        unique_model_names, fuzz.token_set_ratio, scoring_threshold
    )

    # take the one that has the higher score
    unique_models = similarity_sort_ratio["model"].unique().tolist()
    matches = pd.DataFrame(columns=["model", "match", "score"])
    for model in unique_models:

        max_score_sort = similarity_sort_ratio.loc[
            similarity_sort_ratio["model"] == model, "score"
        ].max()
        max_score_set = similarity_set_ratio.loc[
            similarity_set_ratio["model"] == model, "score"
        ].max()

        if max_score_sort > max_score_set:
            idx_max = similarity_sort_ratio.loc[
                similarity_sort_ratio["model"] == model, "score"
            ].idxmax()
            matches = pd.concat(
                [
                    matches,
                    pd.DataFrame(
                        data={
                            "model": [model],
                            "match": [similarity_sort_ratio.loc[idx_max, "match"]],
                            "score": [similarity_sort_ratio.loc[idx_max, "score"]],
                        }
                    ),
                ],
                ignore_index=True,
            )
        else:
            idx_max = similarity_set_ratio.loc[
                similarity_set_ratio["model"] == model, "score"
            ].idxmax()
            matches = pd.concat(
                [
                    matches,
                    pd.DataFrame(
                        data={
                            "model": [model],
                            "match": [similarity_set_ratio.loc[idx_max, "match"]],
                            "score": [similarity_set_ratio.loc[idx_max, "score"]],
                        }
                    ),
                ],
                ignore_index=True,
            )

    # take shortest model name as name for machting groups
    matches["chosen_model_name"] = [
        min([model, match], key=len)
        for model, match in zip(matches["model"], matches["match"])
    ]

    # check if matching makes sense
    matches.groupby(["chosen_model_name", "score"]).agg(
        {"model": ", ".join}
    ).sort_values(["score"], ascending=False)

    matches.drop(
        index=matches.index[matches["score"] < replacing_threshold], inplace=True
    )

    # replace model names with matches
    replace_model_names = dict(
        zip(matches["model"].tolist(), matches["chosen_model_name"].tolist())
    )
    dataset.loc[dataset["manufacturer"] == manufacturer, "model"] = dataset.loc[
        dataset["manufacturer"] == manufacturer, "model"
    ].replace(replace_model_names)

    return dataset

In [ ]:
# match similar model names for same manufacturer
print(
    f"Number of unique model names before matching: {used_car_data['model'].nunique()}"
)

for manufacturer in used_car_data["manufacturer"].unique().tolist():
    used_car_data = find_and_replace_similar_models(
        manufacturer, 80, 100, used_car_data
    )

print(
    f"Number of unique model names after matching: {used_car_data['model'].nunique()}"
)

## Correcting manufacturer names

Some manufacturer names were not correctly split during the web scraping. This relates mostly to manufacturer names which consits of two or more words. 

In [ ]:
used_car_data.loc[
    used_car_data["manufacturer"] == "land", "manufacturer"
] = "land rover"
used_car_data.loc[
    used_car_data["manufacturer"] == "alfa", "manufacturer"
] = "alfa romeo"
used_car_data.loc[used_car_data["manufacturer"] == "vw", "manufacturer"] = "volkswagen"
used_car_data["manufacturer"].unique()

The data set also contains advertisements which come from a test account, which I will just remove from the data set.

In [ ]:
used_car_data.drop(
    index=used_car_data.index[used_car_data["provider"] == "Blocket Testkonto 2"],
    inplace=True,
)
used_car_data.shape

# 2) Handle missing values

The data set contains a lot of missing values for detailed information about the car, such as CO2-emissions, fuel consumption, and dimensions of the car. 

The missing values are can be categorical, such as the type of drive (autmatic or manual), or numeric , such as CO2-emission. 

Categorical values are imputed by choosing the category which occurs most often for the same car model. 

Numerical values are imputed by choosing the median of the same car model if the difference between the mean and median are below a threshold . 

If the data set does not contain a model to take a value from, then no value is imputed.

In [ ]:
def impute_categoricals(dataset, category):
    """Impute missing categorical values of given category in data set.

    Imputation is based on category which appears most often in data set of
    the same model.

    Parameters
    ----------
    dataset : DataFrame
        dataset of used cars
    category : str
        name of column to be imputed

    Returns
    -------
    dataset: DataFrame
        updated dataset of used cars
    """
    new_category = "_".join(["imputed", category])
    dataset[new_category] = dataset[category]

    unique_car_models = dataset.loc[dataset[new_category].isna(), "model"].unique()

    for model in unique_car_models:
        categories_same_model = dataset.loc[
            dataset["model"] == model, new_category
        ].value_counts()

        if len(categories_same_model) > 0:
            idx = dataset.loc[dataset[new_category].isna(), "model"] == model
            dataset.loc[
                idx.index[idx].to_list(), new_category
            ] = categories_same_model.index[0]

    print(
        f"missing values before and after imputing {category}: ",
        f"{dataset[category].isna().sum()} / {dataset[new_category].isna().sum()}",
    )

    return dataset


def impute_numerics_based_on_distribution(dataset, category, threshold):
    """Impute missing numeric values of given category in data set.

    The values are imputed by picking a random value within the range of 2 standard
    deviations around the mean. However, the value is only imputed if the mean and
    median are below a given threshold to ensure that the distribution is close to
    symmetry.

    Parameters
    ----------
    dataset : DataFrame
        dataset of used cars
    category : str
        name of column to be imputed
    threshold : int
        threshold difference between mean and median

    Returns
    -------
    dataset: DataFrame
        updated dataset of used cars
    """
    new_category = "_".join(["imputed", category])
    dataset[new_category] = dataset[category]

    unique_car_models = dataset.loc[dataset[category].isna(), "model"].unique()

    for model in unique_car_models:

        # done so that no warning is printed when there are no values for model type
        if dataset.loc[dataset["model"] == model, category].notnull().any():
            model_mean = np.nanmean(dataset.loc[dataset["model"] == model, category])
            model_median = np.nanmedian(
                dataset.loc[dataset["model"] == model, category]
            )

            if abs(model_mean - model_median) / model_median < threshold:
                idx = dataset.loc[dataset[category].isna(), "model"] == model

                standard_deviation = np.nanstd(
                    dataset.loc[dataset["model"] == model, category]
                )

                dataset.loc[idx.index[idx].to_list(), new_category] = np.random.randint(
                    model_mean - 2 * standard_deviation,
                    model_mean + 2 * standard_deviation,
                    size=len(idx),
                )

    print(
        f"missing values before and after imputing {category}: ",
        f"{dataset[category].isna().sum()} / {dataset[new_category].isna().sum()}",
    )

    return dataset


def impute_numerics_based_on_linear_regression(dataset, category, corr_category):
    """Impute missing numeric values based on linear regression.

    The values are imputed based on a linear correlation to another value.

    Parameters
    ----------
    dataset : DataFrame
        dataset of used cars
    category : str
        name of column to be imputed
    corr_category : str
        name of column / variable which is used for correlation

    Returns
    -------
    dataset: DataFrame
        updated dataset of used cars
    """
    new_category = "_".join(["imputed", category])
    dataset[new_category] = dataset[category]

    prediction_dataset = dataset.copy().dropna(axis=0, subset=new_category)
    prediction_dataset.dropna(axis=0, subset=corr_category, inplace=True)

    x = prediction_dataset[corr_category].to_numpy().reshape((-1, 1))
    y = prediction_dataset[new_category].to_numpy()

    model = LinearRegression().fit(x, y)

    missing_value_idx = dataset.index[
        dataset[new_category].isna() & dataset[corr_category].notna()
    ]

    dataset.loc[missing_value_idx, new_category] = model.predict(
        dataset.loc[missing_value_idx, corr_category].to_numpy().reshape((-1, 1))
    )

    print(
        f"missing values before and after imputing {category}: ",
        f"{dataset[category].isna().sum()} / {dataset[new_category].isna().sum()}",
    )

    return dataset

### Impute categorical values

In [ ]:
used_car_data = impute_categoricals(used_car_data, "fuel")
used_car_data = impute_categoricals(used_car_data, "transmission")
used_car_data = impute_categoricals(used_car_data, "type_of_drive")
used_car_data = impute_categoricals(used_car_data, "emission_class")
used_car_data = impute_categoricals(used_car_data, "car_type")

### Impute numeric values

In [ ]:
sns.lmplot(
    x="co2_emission_g/km",
    y="fuel_consumption_mixed_l_100km",
    data=used_car_data,
    col="car_type",
)
sns.lmplot(
    x="co2_emission_g/km",
    y="fuel_consumption_highway_l_100km",
    data=used_car_data,
    col="car_type",
)
sns.lmplot(
    x="fuel_consumption_mixed_l_100km",
    y="fuel_consumption_highway_l_100km",
    data=used_car_data,
    col="car_type",
)
plt.show()

In [ ]:
used_car_data = impute_numerics_based_on_distribution(used_car_data, "horse_power", 0.1)
used_car_data = impute_numerics_based_on_distribution(
    used_car_data, "fuel_consumption_mixed_l_100km", 0.1
)
used_car_data = impute_numerics_based_on_linear_regression(
    used_car_data, "co2_emission_g/km", "fuel_consumption_mixed_l_100km"
)
used_car_data = impute_numerics_based_on_linear_regression(
    used_car_data, "fuel_consumption_highway_l_100km", "fuel_consumption_mixed_l_100km"
)

In [ ]:
used_car_data.head()

## 3) Remove outliers 

As can be seen, the data set has a great variety of the cars' entry years, prices and mileage. 

For example, the data set seems to 
- include some oldtimers from the 1960s and earlier 
- cars which seem very expensive (greater 5 mio. swedish crown
- cars with an unreasonably high mileage (greater 300000 km)

As I want to focus on cars which are a "realistic" option to buy for someone who is looking for a used car to use daily I will remove these "outliers" of the overall data set. 

To identify the outliers I will determine a lower and upper bound based on the interquartile range.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 3))
ax[0].boxplot(used_car_data["entry_year"])
ax[0].set_title("entry year")
ax[1].boxplot(used_car_data["price_sek"])
ax[1].set_title("price in sek")
ax[2].boxplot(used_car_data["mileage_km"])
ax[2].set_title("mileage in km")

Some car advertisments did not have a price. These advertisments were given a price of 0 swedish krones. As these car advertisments are not insightful I removed them. 

In [ ]:
used_car_data.drop(
    used_car_data.index[used_car_data["price_sek"] <= 0], axis=0, inplace=True
)

In [ ]:
def calculate_whisker_bounds(data):
    """Calculate whisker bounds.

    Parameters
    ----------
    data : Series
        data

    Returns
    -------
    lower_bound: int
        end of lower whisker
    upper_bound: int
        end of upper whisker
    """
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR

    return (lower_bound, upper_bound)


def remove_outliers(dataset, category):
    """Remove outliers in given category.

    Parameters
    ----------
    dataset : DataFrame
        data set
    category : str
        category in which outliers shall be removed

    Returns
    -------
    dataset : DataFrame
        updated data set
    """
    lower_bound, upper_bound = calculate_whisker_bounds(used_car_data[category])

    index_out_of_range_entry_years = dataset.index[
        (dataset[category] > upper_bound) | (dataset[category] < lower_bound)
    ]

    number_cars = dataset.shape[0]
    dataset.drop(index_out_of_range_entry_years, axis=0, inplace=True)

    print(
        f"number of used cars before and after removing outliers from {category}: ",
        f"{number_cars} / {dataset.shape[0]}",
    )

    return dataset

In [ ]:
used_car_data = remove_outliers(used_car_data, "entry_year")
used_car_data = remove_outliers(used_car_data, "price_sek")
used_car_data = remove_outliers(used_car_data, "mileage_km")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 3))
ax[0].boxplot(used_car_data["entry_year"])
ax[0].set_title("entry year")
ax[1].boxplot(used_car_data["price_sek"])
ax[1].set_title("price in sek")
ax[2].boxplot(used_car_data["mileage_km"])
ax[2].set_title("mileage in km")

## Remove any non necessary information

In [ ]:
used_car_data.drop(
    [
        "note",
        "engine_size_ccm",
        "top_speed_km_h",
        "length_mm",
        "width_mm",
        "height_mm",
        "load_capacity_kg",
        "empty_weight_kg",
        "total_weight_kg",
        "number_of_seats",
        "url",
    ],
    axis=1,
    inplace=True,
)

# remove cars which could not be classified by their type
used_car_data.drop(
    index=used_car_data.index[used_car_data["imputed_car_type"].isna()], inplace=True
)

used_car_data.shape

# Save cleaned data set

In [ ]:
used_car_data.reset_index(drop=True, inplace=True)
used_car_data.to_csv("../data/cleaned_used_car_dataset.csv")